# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# 수정사항
- 2023.03.21
    - item 2 : EG
        - 1. 개별 차량에 대한 지역코드를 부여하여, 지역마다 얼마나 차량들이 존재하는지 구분이 필요한 것 같음

    이 외에도 내일 오전에 동양에서 가시화와 관련된 회의에서 추가적으로 필요하다고 판단되는 사항에 대해 추가적으로 분석이 필요할 것 같음

- 2023.03.22
    - result파일에 'EG' 컬럼 추가

- 2023.03.24
    - result파일에 '배인번호_수정', '등급_수정', 'DPF유무_수정' 컬럼 추가

# fold path

In [3]:
drive = 'D'

In [4]:
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'
df_fold = f'{drive}:/data/big2/BD1/df'

# Load
- 등록&제원 -> result 4등급(연료 : 경유, 휘발유, LPG)

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [ ]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')

In [ ]:
# exasol db
# 8.6s
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
# car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG;")

In [ ]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898814 entries, 0 to 1898813
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   VIN                object
 1   BSPL_STDG_CD       int64 
 2   EXHST_GAS_GRD_CD   object
 3   EXHST_GAS_CERT_NO  object
 4   VHCL_ERSR_YN       object
 5   MANP_MNG_NO        object
 6   YRIDNW             int64 
 7   VHCTY_CD           object
 8   PURPS_CD2          object
 9   FRST_REG_YMD       int64 
dtypes: int64(3), object(7)
memory usage: 144.9+ MB


In [ ]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
}

In [ ]:
carr = car.rename(columns=car_ch_col)

In [ ]:
carr.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [ ]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [ ]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [ ]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [ ]:
# 3m 34.9s
att = wb.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64316863 entries, 0 to 64316862
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   VIN                     object 
 1   FDRM_INSP_INSP_MTHD_CD  object 
 2   FDRM_INSP_KND_CD        object 
 3   FDRM_INSP_JGMT          object 
 4   FDRM_NLOD_SMO_MSTVL1    float64
 5   FDRM_NLOD_SMO_MSTVL2    float64
 6   FDRM_NLOD_SMO_MSTVL3    float64
 7   FDRM_NLOD_SMO_JT_YN1    object 
 8   FDRM_INSP_YMD           int64  
 9   FDRM_DRVNG_DSTNC        float64
 10  FDRM_NLOD_SMO_PRMVL1    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.3+ GB


In [ ]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [ ]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [ ]:
# 3m 34.9s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM,  FROM STD_BJCD_INFO;")
code.info()

# 전처리

## 중복 차대번호 제거

In [ ]:
carr.shape, len(carr['차대번호'].unique())

In [ ]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicate('차대번호').reset_index(drop=True)
carr.shape

## 등록정보 말소 제거

In [ ]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(26205893, 10)

## 배출가스등급 코드 변환

In [ ]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carm['배출가스등급'] = carm['배출가스등급'].replace(grd_dict)
carm['배출가스등급'].unique()

array(['3', '5', '4', 'X', '2', '1'], dtype=object)

## 차종 코드 변환

In [ ]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carm['차종'] = carm['차종'].replace(cd_dict)
carm['차종'].unique()

array(['승합', '승용', '화물', '특수', nan], dtype=object)

## 용도 코드 변환

In [ ]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carm['용도'] = carm['용도'].replace(purps_dict)
carm['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 연료 코드 변환

In [ ]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '전기', '알코올', 'CNG 하이브리드', '등유', '태양열', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [ ]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [ ]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(26205893, 16)

## 저감장치구분 코드 변환

In [ ]:
# 4m 14.3s
attr['저감장치구분'].unique()

(34114272, 10)

## 검사종류 코드 변환

In [ ]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['종합검사', '종합검사(경과)', '1부재검사(종합)', '2부재검사(종합)', '정기검사', '재검사(1부재검사)',
       '재검사(2부재검사)', '정기(경과)검사', '배출재검사(구변)', nan, '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       'A21C01', '특정검사', '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사',
       '특정재검사(관능)', '특정검사(경과)', '정밀검사(경과)', 'A21R03', 'A21R05', 'A21R06',
       '관능재검사(임시)', 'A21E03'], dtype=object)

## 저감장치 부착 유무

In [ ]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [ ]:
attr['DPF_YN'].value_counts(dropna=False)

## 등록&제원&저감이력 병합

In [ ]:
# 2m 0.5s
df = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
df.shape

(26205893, 25)

In [ ]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '정기정밀'],
      dtype='object')

# 전처리

## 연료 확인

In [37]:
df['연료'].value_counts(dropna=False)

연료
경유             1149256
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: count, dtype: int64

## 연료 컬럼 추가

In [38]:
# 1149256
rw1 = df.loc[df['연료'] == '경유'].reset_index(drop=True)
rw1.shape

(1149256, 60)

In [39]:
# 348419
rw2 = df.loc[(df['연료'] == '휘발유') | (df['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)
rw2.shape

(348419, 60)

In [40]:
# 3
rw_rest = df.loc[(df['연료'] == '알코올') | (df['연료'] == 'CNG(압축천연가스)') | (df['연료'] == '기타연료')].reset_index(drop=True)
rw_rest.shape

(3, 60)

In [43]:
rw1['fuel'] = '경유'
rw2['fuel'] = '휘발유_가스'

## 경유, 휘발유_가스, 나머지 병합

In [50]:
df1 = pd.concat([rw1, rw2, rw_rest], ignore_index=True)
df1.shape

(1497678, 61)

In [51]:
df1['fuel'].value_counts(dropna=False)

fuel
경유        1149256
휘발유_가스     348419
NaN             3
Name: count, dtype: int64

In [52]:
df1['DPF_YN'].value_counts(dropna=False)

DPF유무_수정
무       879332
NaN     348422
유       257763
확인불가     12161
Name: count, dtype: int64

# 분석

## EG 분류

In [55]:
grade_list = []
for f, y, cy, e in tqdm(df1[['fuel', '제작일자', '연식', 'DPF_YN']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1497678/1497678 [00:02<00:00, 524302.13it/s]


1497678

In [59]:
df['Grade'] = grade_list

## [출력] 4등급

In [66]:
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '검사방법', '검사종류', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '정기정밀', 'fuel', 'Grade'],
      dtype='object')

In [79]:
df2 = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '최초등록일', 
    '차량연식', 
    '제작일자',
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도명', 
    '시군구명', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF_YN',
    'Grade',
    '무부하매연측정치1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    ]]
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '차종분류', '차종유형',
       '연료', '법정동코드', '시도명', '시군구명', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3'],
      dtype='object')

In [ ]:
today_date = datetime.today().strftime("%Y%m")
today_date

In [80]:
df2['기준연월'] = today_date

C:\Users\kbjung\AppData\Local\Temp\ipykernel_14412\2157983385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['기준연월'] = '2022.12'


In [81]:
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '차종분류', '차종유형',
       '연료', '법정동코드', '시도명', '시군구명', '배인번호_수정', '등급_수정', 'DPF유무_수정', 'Grade',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '기준연월'],
      dtype='object')

In [82]:
df3 = df2[[
    '기준연월', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '최초등록일', 
    '차량연식', 
    '제작일자', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '배출가스등급', 
    'DPF_YN',
    'Grade', 
    '무부하매연측정치1',
    '무부하매연측정치2',
    '무부하매연측정치3',
    ]]
df3.columns

Index(['기준연월', '차대번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '차종분류',
       '차종유형', '연료', '법정동코드', '시도명', '시군구명', '배인번호_수정', '등급_수정', 'DPF유무_수정',
       'Grade', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3'],
      dtype='object')

In [83]:
ch_col_dict = {
                '기준연월':'CRTR_YM',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '최초등록일':'FRST_REG_YMD', 
                '연식':'YRIDNW', 
                '제작일자':'FBCTN_YMD',
                '차종분류':'VHCTY_CL_CD',
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도명':'CTPV_NM',
                '시군구명':'SGG_NM',
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD', 
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1',
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2',
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3',
                }

In [84]:
df3 = df3.rename(columns=ch_col_dict)
df3.columns

Index(['CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2',
       'FRST_REG_YMD', 'YRIDNW', 'FBCTN_YMD', 'VHCTY_CL_CD', 'VHCTY_TY',
       'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'EXHST_GAS_CERT_NO_MOD',
       'EXHST_GAS_GRD_CD_MOD', 'DPF_MNTNG_YN', 'GRD4_MLSFC',
       'NOLOD_SMO_MEVLU3', '무부하매연측정치2', '무부하매연측정치3'],
      dtype='object')

In [74]:
# about 9.7s
# 4등급 세분류 결과
df3.to_csv(os.path.join(df_fold, f'BD1-2_STD_BD_GRD4_MLSFC_RSLT.csv'), index=False)

## [출력] 4등급 + 3등급(등급_수정) + 5등급(등급_수정)

In [77]:
total = pd.concat([df, g3_g5], ignore_index=True)
total.shape

(1502235, 62)

In [78]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', '검사방법', '검사종류', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '정기정밀', 'fuel', 'Grade'],
      dtype='object')

In [79]:
total = total.drop(['검사방법', '검사종류', '검사판정', '산소값', '이산화탄소값', '무부하매연판정1',
       '무부하매연판정2', '무부하매연판정3', '무부하매연판정4', '무부하매연판정5', '무부하매연판정6', '무부하매연허용치1',
       '무부하매연허용치2', '무부하매연허용치3', '무부하매연허용치4', '무부하매연허용치5', '무부하매연허용치6',
       '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연측정치4', '무부하매연측정치5',
       '무부하매연측정치6', '정기정밀', 'fuel'], axis=1)
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade'],
      dtype='object')

In [80]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.06.12_PM.03.50.44'

In [81]:
# # about 21.8s
# total.to_csv(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result(전체)_Grade_추가({today_date}).csv'), index=False, encoding='cp949')

In [82]:
# # about 14m 7.3s
# n = 700000
# total.iloc[:n].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result(전체)_Grade_추가({today_date})01.xlsx'), index=False)
# total.iloc[n:].to_excel(os.path.join(raw_fold, f'4등급_차량_리스트_양식_result(전체)_Grade_추가({today_date})02.xlsx'), index=False)

## 시도, 연도별 차량 대수

In [83]:
total['최초등록일'].describe()

count    1502235.00
mean    20056945.62
std        68680.98
min         1993.00
25%     20060824.00
50%     20081217.00
75%     20100512.00
max     20221125.00
Name: 최초등록일, dtype: float64

In [84]:
total.shape

(1502235, 37)

In [85]:
total.loc[(total['최초등록일'] >= 20190101) & (total['최초등록일'] < 20230101)].shape

(144, 37)

In [86]:
total2 = total.copy()

In [87]:
total2['최초등록일'] = total2['최초등록일'].astype('str')

In [88]:
total2.loc[total2['최초등록일'].str.len() != 8]

,차대번호,차량번호,제원관리번호,차종,용도,최초등록일,연식,제작일자,검사유효일,배출가스인증번호,...,일치유무,인증번호 수정 유무,배인번호_수정,등급_수정,DPF유무_수정,배출가스,등급2,DPF,비고,Grade
1172636,KMHJF31JPPU555457,경기34머9199,10022700140000,승용,자가용,1993,1993,1993.00,20031014.00,KMY-HD-04,...,NaN,NaN,NaN,4.00,NaN,NaN,NaN,NaN,NaN,B
1260821,KLAER1931TB044115,강원27누7569,10002200860001,승용,자가용,9960617,1996,9960617.00,20020616.00,NaN,...,NaN,NaN,NaN,4.00,NaN,NaN,NaN,NaN,NaN,B


### 최초등록일 수정

In [89]:
total2.loc[total2['최초등록일'] == '1993', '최초등록일'] = '19930101'
total2.loc[total2['최초등록일'] == '9960617', '최초등록일'] = '19960617'

In [90]:
total2.loc[total2['최초등록일'].str.len() != 8].shape

(0, 37)

In [91]:
total2['최초등록일_년'] = total2['최초등록일'].str[:4]
total2['최초등록일_월'] = total2['최초등록일'].str[4:6]
total2['최초등록일_일'] = total2['최초등록일'].str[6:8]

In [92]:
total2[['최초등록일', '최초등록일_년', '최초등록일_월', '최초등록일_일']].head()

,최초등록일,최초등록일_년,최초등록일_월,최초등록일_일
0,20060208,2006,02,08
1,20060228,2006,02,28
2,20060105,2006,01,05
3,20060105,2006,01,05
4,20060630,2006,06,30


In [93]:
total2['최초등록일_년월'] = total2['최초등록일_년'] + '.' + total2['최초등록일_월']
total2['최초등록일_년월'].head()

0    2006.02
1    2006.02
2    2006.01
3    2006.01
4    2006.06
Name: 최초등록일_년월, dtype: object

In [94]:
total2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '최초등록일_년', '최초등록일_월', '최초등록일_일', '최초등록일_년월'],
      dtype='object')

### 시군구명 앞쪽 지역명만 남기기

In [95]:
total2.loc[50:55, ['시도명', '시군구명']]

,시도명,시군구명
50,경기도,용인시 기흥구
51,경기도,안양시 동안구
52,전라북도,전주시 덕진구
53,경기도,하남시
54,대구광역시,수성구
55,울산광역시,북구


In [96]:
total2['시군구명_수정'] = total2['시군구명'].str.split(' ').str[0]

In [97]:
total2.loc[50:55, ['시도명', '시군구명', '시군구명_수정']]

,시도명,시군구명,시군구명_수정
50,경기도,용인시 기흥구,용인시
51,경기도,안양시 동안구,안양시
52,전라북도,전주시 덕진구,전주시
53,경기도,하남시,하남시
54,대구광역시,수성구,수성구
55,울산광역시,북구,북구


In [99]:
num_car_by_local = total2.groupby(['연료', '시도명', '시군구명_수정', '최초등록일_년', '최초등록일_월'])['차대번호'].count().reset_index()
num_car_by_local = num_car_by_local.rename(columns={'차대번호':'차량대수'})
num_car_by_local

,연료,시도명,시군구명_수정,최초등록일_년,최초등록일_월,차량대수
0,CNG(압축천연가스),전라남도,신안군,2005,04,1
1,LPG(액화석유가스),강원도,강릉시,1990,03,1
2,LPG(액화석유가스),강원도,강릉시,1992,04,1
3,LPG(액화석유가스),강원도,강릉시,1993,05,1
4,LPG(액화석유가스),강원도,강릉시,1993,07,1
...,...,...,...,...,...,...
71439,휘발유,충청북도,충주시,2008,08,1
71440,휘발유,충청북도,충주시,2009,02,1
71441,휘발유,충청북도,충주시,2009,03,1
71442,휘발유,충청북도,충주시,2013,05,1


#### 출력 

In [129]:
# # about 
# num_car_by_local.to_excel(os.path.join(analysis_fold, '통계_result_시군구_월별_차량수.xlsx'), index=False)

### 시도 시군구별 차량 현황

In [100]:
total2.shape

(1502235, 42)

In [101]:
total2['최초등록일'].astype('int').describe()

count    1502235.00
mean    20056965.54
std        66192.91
min     19650209.00
25%     20060824.00
50%     20081217.00
75%     20100512.00
max     20221125.00
Name: 최초등록일, dtype: float64

In [102]:
total2.loc[total2['최초등록일'] < '20220601'].shape

(1502232, 42)

In [103]:
total2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '최초등록일_년', '최초등록일_월', '최초등록일_일', '최초등록일_년월',
       '시군구명_수정'],
      dtype='object')

In [132]:
num_car_by_local2 = total2.loc[total2['최초등록일'] < '20220601'].groupby(['연료', '시도명', '시군구명_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'차량대수'})
num_car_by_local2

,연료,시도명,시군구명_수정,차량대수
0,CNG(압축천연가스),전라남도,신안군,1
1,LPG(액화석유가스),강원도,강릉시,66
2,LPG(액화석유가스),강원도,고성군,13
3,LPG(액화석유가스),강원도,동해시,183
4,LPG(액화석유가스),강원도,삼척시,19
...,...,...,...,...
688,휘발유,충청북도,증평군,306
689,휘발유,충청북도,진천군,750
690,휘발유,충청북도,청원군,3
691,휘발유,충청북도,청주시,5094


In [133]:
num_car_by_local2['연월'] = '2022.06'
num_car_by_local2

,연료,시도명,시군구명_수정,차량대수,연월
0,CNG(압축천연가스),전라남도,신안군,1,2022.06
1,LPG(액화석유가스),강원도,강릉시,66,2022.06
2,LPG(액화석유가스),강원도,고성군,13,2022.06
3,LPG(액화석유가스),강원도,동해시,183,2022.06
4,LPG(액화석유가스),강원도,삼척시,19,2022.06
...,...,...,...,...,...
688,휘발유,충청북도,증평군,306,2022.06
689,휘발유,충청북도,진천군,750,2022.06
690,휘발유,충청북도,청원군,3,2022.06
691,휘발유,충청북도,청주시,5094,2022.06


In [134]:
# num_car_by_local2.to_excel(os.path.join(analysis_fold, 'BD1_통계_4등급차_시도_시군구별_차량현황_2022년6월.xlsx'), index=False)

### 시도, 시군구, 월, 월 기준 차량대수 및 말소 대수

#### [로드] 말소정보_전처리완료

In [106]:
# about 19.4s
name = '말소정보_전처리완료(2023.05.24)'
file_name = f'{name}.csv'
erase = pd.read_csv(os.path.join(ods_fold, file_name), low_memory=False)
erase.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642251 entries, 0 to 3642250
Data columns (total 24 columns):
 #   Column       Dtype  
---  ------       -----  
 0   차량관리번호       object 
 1   본거지법정동코드     int64  
 2   차종_등록정보      object 
 3   차대번호         object 
 4   차량연식         int64  
 5   제원관리번호       object 
 6   배출가스인증번호     object 
 7   배출가스등급       object 
 8   차량말소YN_등록정보  object 
 9   차명           object 
 10  연료           object 
 11  등급           object 
 12  차량말소YN_변경이력  object 
 13  변경일자         float64
 14  변경구분코드       float64
 15  법정동코드        int64  
 16  시도           object 
 17  시구군          object 
 18  읍면동          object 
 19  리            object 
 20  지역3          float64
 21  폐지여부         object 
 22  대기관리권역       object 
 23  대기관리권역YN     object 
dtypes: float64(3), int64(3), object(18)
memory usage: 666.9+ MB


In [107]:
erase1 = erase[['차대번호', '연료', '시도', '시구군', '등급', '변경일자']]
erase1.shape

(3642251, 6)

In [108]:
erase1['등급'].value_counts(dropna=False)

등급
3.0    1198427
5.0    1189962
2.0     783830
4.0     366937
1.0      88800
X        14263
NaN         32
Name: count, dtype: int64

In [138]:
erase1['변경일자'].isnull().sum()

0

In [109]:
erase2 = erase1.loc[erase1['등급'] == '4.0'].reset_index(drop=True)
erase2.shape

(366937, 6)

In [110]:
erase2['변경일자'] = erase2['변경일자'].astype('str')

In [111]:
erase2['변경일자'].head()

0    20220628.0
1    20220628.0
2    20220628.0
3    20220628.0
4    20220628.0
Name: 변경일자, dtype: object

In [112]:
erase2['변경일자_년'] = erase2['변경일자'].str[:4]
erase2['변경일자_월'] = erase2['변경일자'].str[4:6]
erase2['변경일자_일'] = erase2['변경일자'].str[6:8]

In [113]:
erase2['변경일자_년월'] = erase2['변경일자_년'] + '.' + erase2['변경일자_월']

In [114]:
erase2.loc[erase2['시구군'].str.split(' ').str.len() != 1].head()

,차대번호,연료,시도,시구군,등급,변경일자,변경일자_년,변경일자_월,변경일자_일,변경일자_년월
0,KNADE22437S225673,경유,경기도,성남시 수정구,4.0,20220628.0,2022,06,28,2022.06
2,KMHSH81VP8U393442,경유,경기도,용인시 수지구,4.0,20220628.0,2022,06,28,2022.06
8,KMHEN41BPXA100311,휘발유,경기도,고양시 일산서구,4.0,20220628.0,2022,06,28,2022.06
9,KNADE24427S291651,경유,충청북도,청주시 청원구,4.0,20220628.0,2022,06,28,2022.06
12,KNHMB76438S272945,경유,경기도,고양시 일산서구,4.0,20220628.0,2022,06,28,2022.06


In [115]:
erase2['시구군'] = erase2['시구군'].str.split(' ').str[0]
erase2.loc[erase2['시구군'].str.split(' ').str.len() != 1].head()

,차대번호,연료,시도,시구군,등급,변경일자,변경일자_년,변경일자_월,변경일자_일,변경일자_년월


In [139]:
grp_erase = erase2.loc[erase2['변경일자_년'] == '2022'].groupby(['변경일자_년월', '연료', '시도', '시구군'])['차대번호'].count().reset_index()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수'})
grp_erase = grp_erase.sort_values(['시도', '시구군', '변경일자_년월'])
grp_erase

,변경일자_년월,연료,시도,시구군,말소차량대수
0,2022.01,LPG(액화석유가스),강원도,강릉시,1
106,2022.01,경유,강원도,강릉시,26
353,2022.01,휘발유,강원도,강릉시,11
671,2022.02,경유,강원도,강릉시,38
917,2022.02,휘발유,강원도,강릉시,6
...,...,...,...,...,...
2256,2022.04,휘발유,충청북도,충주시,6
2609,2022.05,경유,충청북도,충주시,47
2829,2022.05,휘발유,충청북도,충주시,7
3168,2022.06,경유,충청북도,충주시,44


In [140]:
grp_erase.columns

Index(['변경일자_년월', '연료', '시도', '시구군', '말소차량대수'], dtype='object')

In [141]:
num_car_by_local2.columns

Index(['연료', '시도명', '시군구명_수정', '차량대수', '연월'], dtype='object')

In [142]:
num_car_by_month2022 = grp_erase.merge(num_car_by_local2, left_on=['변경일자_년월', '시도', '시구군', '연료'], right_on=['연월', '시도명', '시군구명_수정', '연료'], how='left')
num_car_by_month2022 = num_car_by_month2022.drop(['시도명', '시군구명_수정', '연월'], axis=1)
num_car_by_month2022 = num_car_by_month2022[['변경일자_년월', '시도', '시구군', '연료', '말소차량대수', '차량대수']]
num_car_by_month2022 = num_car_by_month2022.sort_values(['연료', '시도', '시구군', '변경일자_년월'])
num_car_by_month2022

,변경일자_년월,시도,시구군,연료,말소차량대수,차량대수
0,2022.01,강원도,강릉시,LPG(액화석유가스),1,NaN
5,2022.03,강원도,강릉시,LPG(액화석유가스),3,NaN
12,2022.06,강원도,강릉시,LPG(액화석유가스),1,66.00
33,2022.05,강원도,동해시,LPG(액화석유가스),1,NaN
36,2022.06,강원도,동해시,LPG(액화석유가스),1,183.00
...,...,...,...,...,...,...
3369,2022.02,충청북도,충주시,휘발유,8,NaN
3371,2022.03,충청북도,충주시,휘발유,7,NaN
3373,2022.04,충청북도,충주시,휘발유,6,NaN
3375,2022.05,충청북도,충주시,휘발유,7,NaN


In [145]:
uni = num_car_by_month2022[['시도', '시구군', '연료']].drop_duplicates(['시도', '시구군', '연료'])
uni.head()

,시도,시구군,연료
0,강원도,강릉시,LPG(액화석유가스)
33,강원도,동해시,LPG(액화석유가스)
55,강원도,속초시,LPG(액화석유가스)
91,강원도,영월군,LPG(액화석유가스)
100,강원도,원주시,LPG(액화석유가스)


In [149]:
ttemp = pd.DataFrame()
for si, sgg, fuel in uni.values:
    temp = pd.DataFrame({'변경일자_년월':['2022.01', '2022.02', '2022.03', '2022.04', '2022.05', '2022.06'], '시도':[si] * 6, '시구군':[sgg]*6, '연료':[fuel]*6})
    ttemp = pd.concat([ttemp, temp], ignore_index=True)
ttemp.head

<bound method NDFrame.head of       변경일자_년월    시도  시구군           연료
0     2022.01   강원도  강릉시  LPG(액화석유가스)
1     2022.02   강원도  강릉시  LPG(액화석유가스)
2     2022.03   강원도  강릉시  LPG(액화석유가스)
3     2022.04   강원도  강릉시  LPG(액화석유가스)
4     2022.05   강원도  강릉시  LPG(액화석유가스)
...       ...   ...  ...          ...
4171  2022.02  충청북도  충주시          휘발유
4172  2022.03  충청북도  충주시          휘발유
4173  2022.04  충청북도  충주시          휘발유
4174  2022.05  충청북도  충주시          휘발유
4175  2022.06  충청북도  충주시          휘발유

[4176 rows x 4 columns]>

In [170]:
num_car = ttemp.merge(num_car_by_month2022, on=['변경일자_년월', '시도', '시구군', '연료'], how='left')
num_car.head(6)

,변경일자_년월,시도,시구군,연료,말소차량대수,차량대수
0,2022.01,강원도,강릉시,LPG(액화석유가스),1.00,NaN
1,2022.02,강원도,강릉시,LPG(액화석유가스),NaN,NaN
2,2022.03,강원도,강릉시,LPG(액화석유가스),3.00,NaN
3,2022.04,강원도,강릉시,LPG(액화석유가스),NaN,NaN
4,2022.05,강원도,강릉시,LPG(액화석유가스),NaN,NaN
5,2022.06,강원도,강릉시,LPG(액화석유가스),1.00,66.00


In [171]:
num_car.loc[2, '말소차량대수']

3.0

In [172]:
num_car.shape[0] // 6

696

In [173]:
num_car[['말소차량대수', '차량대수']] = num_car[['말소차량대수', '차량대수']].fillna(0)
num_car[['말소차량대수', '차량대수']].isnull().sum()

말소차량대수    0
차량대수      0
dtype: int64

In [174]:
for i in range(num_car.shape[0] // 6):
    if np.isnan(num_car.loc[(i+1)*6 - 1, '차량대수']) == True:
        num_car.loc[(i+1)*6 - 1, '차량대수'] = 0
    num_car.loc[(i+1)*6 - 2, '차량대수'] = num_car.loc[(i+1)*6 - 1, '차량대수'] + num_car.loc[(i+1)*6 - 1, '말소차량대수']
    num_car.loc[(i+1)*6 - 3, '차량대수'] = num_car.loc[(i+1)*6 - 2, '차량대수'] + num_car.loc[(i+1)*6 - 2, '말소차량대수']
    num_car.loc[(i+1)*6 - 4, '차량대수'] = num_car.loc[(i+1)*6 - 3, '차량대수'] + num_car.loc[(i+1)*6 - 3, '말소차량대수']
    num_car.loc[(i+1)*6 - 5, '차량대수'] = num_car.loc[(i+1)*6 - 4, '차량대수'] + num_car.loc[(i+1)*6 - 4, '말소차량대수']
    num_car.loc[(i+1)*6 - 6, '차량대수'] = num_car.loc[(i+1)*6 - 5, '차량대수'] + num_car.loc[(i+1)*6 - 5, '말소차량대수']

In [175]:
num_car.head()

,변경일자_년월,시도,시구군,연료,말소차량대수,차량대수
0,2022.01,강원도,강릉시,LPG(액화석유가스),1.00,70.00
1,2022.02,강원도,강릉시,LPG(액화석유가스),0.00,70.00
2,2022.03,강원도,강릉시,LPG(액화석유가스),3.00,67.00
3,2022.04,강원도,강릉시,LPG(액화석유가스),0.00,67.00
4,2022.05,강원도,강릉시,LPG(액화석유가스),0.00,67.00


In [176]:
num_car.tail(6)

,변경일자_년월,시도,시구군,연료,말소차량대수,차량대수
4170,2022.01,충청북도,충주시,휘발유,8.00,1471.00
4171,2022.02,충청북도,충주시,휘발유,8.00,1463.00
4172,2022.03,충청북도,충주시,휘발유,7.00,1456.00
4173,2022.04,충청북도,충주시,휘발유,6.00,1450.00
4174,2022.05,충청북도,충주시,휘발유,7.00,1443.00
4175,2022.06,충청북도,충주시,휘발유,7.00,1436.00


### [출력] # STD_BD_GRD4_RGN_CURSTT

In [177]:
# STD_BD_GRD4_RGN_CURSTT
num_car.to_excel(os.path.join(analysis_fold, 'BD1_통계_4등급차_시도_시군구_2022년_월별_차량현황.xlsx'), index=False)

# 코드 마지막